1. Add headers = ["Document Source","Requirement Number","CTBC Requirement Head","CTBC Description","TCS BaNCS Component","Module Name","TCS understanding (General Requirement Description)","Fitement Analysis (Standard/Partial Gap/Gap)","Customisattion Efforts Estimate (XL/L/M/S)","Solution Summary","Remark, If Any"]
2. write word content to excel
    - save all the requirement number
    - write original content (split requirement number and description and add 7 empty string elements at last)
    - add requirement head
3. excel 第一個 sheet 改 style (all cells with borders, headers with #002060 as background and #FFFFFF as font color and bold font, content celss with #E7E6E6)
4. 加 table 到 excel sheets (add borders)


In [1]:
import docx
import openpyxl
import pandas as pd
import re

In [2]:
file_name = 'Navi-F2-030-UR-1.14-庫存管理_20221028'
doc = docx.Document(file_name + '.docx')
all_par = doc.paragraphs
len(all_par)

116

In [6]:
num_list = []
doc_source = re.match("^[A-Za-z0-9]*-[A-Za-z0-9]*-[A-Za-z0-9]*-[A-Za-z0-9]*-[A-Za-z0-9_\.]*", file_name)[0]
list = []
list.append(["Document Source","Requirement Number","CTBC Requirement Head","CTBC Description","TCS BaNCS Component","Module Name","TCS understanding (General Requirement Description)","Fitement Analysis (Standard/Partial Gap/Gap)","Customisattion Efforts Estimate (XL/L/M/S)","Solution Summary","Remark, If Any"])
for par in all_par:
    text = par.text.strip()
    if (text != ''):
        number = re.match("^[1-9-\.]+[\.]?", text)
        content = re.split("^([1-9-\.]+[\.]?)", text)
        curr = []
        if number:
            num_list.append(number[0])
            curr = [content[1], "", content[2].strip()]
        else:
            curr = ["", "", content[0].strip()]
        list.append([doc_source] + curr + ["","","","","","","",""])
df = pd.DataFrame(list)

In [5]:
df

,0,1,2,3,4,5,6,7,8,9,10,11
0,Document Source,Requirement Number,CTBC Requirement Head,CTBC Description,TCS BaNCS Component,Module Name,TCS understanding (General Requirement Descrip...,Fitement Analysis (Standard/Partial Gap/Gap),Customisattion Efforts Estimate (XL/L/M/S),Solution Summary,"Remark, If Any",None
1,Navi-F2-030-UR-1.14,,,Navi-F2-030-UR-1.14- Gold inventory position m...,,,,,,,,
2,Navi-F2-030-UR-1.14,,,一、Gold Inventory Position Management_Requirements,,,,,,,,
3,Navi-F2-030-UR-1.14,1.,,Requirements__Gold inventory position management,,,,,,,,
4,Navi-F2-030-UR-1.14,1-1.,,Gold inventory position management- Requiremen...,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...
80,Navi-F2-030-UR-1.14,7.,,Reports (Samples+Specs),,,,,,,,
81,Navi-F2-030-UR-1.14,,,NA,,,,,,,,
82,Navi-F2-030-UR-1.14,8.,,Batch,,,,,,,,
83,Navi-F2-030-UR-1.14,,,三、Gold Inventory Position Management _Future R...,,,,,,,,


In [5]:
for table in doc.tables:
    for row in table.rows:
        for cell in row.cells:
            print (cell.text)
    print('-------')

Version
Requirement no.
Date
Maintenance/
View Description
Maintenance/
View Staff
Review/Reply Date
Review/
Reply Staff
1.0
Transformation
2022/03/28
Initial Version
謝志鵬
2022/03/29
張哲宇
1.1
IN Review
2022/06/07
<金品>No opinion
<作規> No opinion
李櫻怡
熊宜萍
2022/06/10
張哲宇
謝志鵬
1.2
Enhance Document
2022/10/17
1. Enhanced system screen and flow chart
2. Revise the text description, especially the description of the requirements content (item requirements description + future requirements)
3. If it is the part that only supplements the explanation for the manufacturer to understand, move it to the current situation explanation section of each book
4. Due to moving back and forth, some parts are rearranged, but the main structure remains unchanged
張哲宇
謝志鵬
李櫻怡
姚基仁
2022/10/27
張哲宇
謝志鵬
-------
Item
Fields
Description
1
Function
Options:
01-Claiming bank operation;
02-Inventory inquiry
Cannot input other options，display method: code + Chinese description
2
Branch

3
Start Date

4
End Date

5
Print Type
